In [69]:
#importações das bibliotecas
import pandas as pd
import requests


## Campeonato brasileiro 2024

In [70]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A')

# Informações do campeonato de 2024
tabelas2024 = pd.read_html(seriaA2024.text)
tabela_classificação2024 = tabelas2024[6]
tabela_jogos2024 = tabelas2024[7]

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\3065207342.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2024 = pd.read_html(seriaA2024.text)


In [71]:
#criando uma lista com os nomes dos times
nomes_times_2024 = list(tabela_jogos2024["Casa \ Fora"])
#criando uma lista com as siglas dos times
sigla_times_2024 = list(tabela_jogos2024.columns)
sigla_times_2024.pop(0)

#criando um dicionário com a sigla dos times
apelidos_times_2024 = dict(zip(sigla_times_2024, nomes_times_2024))


In [72]:
# Ajustar a tabela de jogos
tabela_jogos2024_ajustada = tabela_jogos2024.set_index("Casa \ Fora")

# Desempilhar a tabela
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.unstack().reset_index()

# Renomear as colunas
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

# Função para ajustar o nome dos times
def ajustar_apelido(linha, apelido):
    time = linha["Fora"]
    return apelido[time]

# Ajustar o nome dos times aplicando a função
tabela_jogos2024_ajustada["Fora"] = tabela_jogos2024_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times_2024,))
# Remover jogos entre o mesmo time
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Fora"] != tabela_jogos2024_ajustada["Casa"]]


In [73]:
tabela_jogos2024_ajustada["Placar"] = tabela_jogos2024_ajustada["Placar"].fillna("A jogar")

jogos_realizados = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = tabela_jogos2024_ajustada[~tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = jogos_faltantes.drop(columns=["Placar"])

# Separando a coluna Placar em gols_casa e gols_fora
jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)
jogos_realizados = jogos_realizados.drop(columns=["Placar"])

"""
# Media de gols feitos dentro de casa de um time
# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa
media_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})

# Media de gols feitos e sofridos fora de casa de um time
media_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})
"""


C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\1598105354.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\1598105354.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)


'\n# Media de gols feitos dentro de casa de um time\n# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa\nmedia_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)\nmedia_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})\n\n# Media de gols feitos e sofridos fora de casa de um time\nmedia_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)\nmedia_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})\n'

## Campeonato Brasileiro 2023

In [74]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2023_-_S%C3%A9rie_A')

tabelas2023 = pd.read_html(seriaA2024.text)
tabela_jogos = tabelas2023[6]

nomes_times = list(tabela_jogos["Casa \ Fora"])
sigla_times = list(tabela_jogos.columns)
sigla_times.pop(0)

apelidos_times = dict(zip(sigla_times, nomes_times))

tabela_jogos_ajustada = tabela_jogos.set_index("Casa \ Fora")
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

tabela_jogos_ajustada["Fora"] = tabela_jogos_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times,))
tabela_jogos_ajustada = tabela_jogos_ajustada[tabela_jogos_ajustada["Casa"] != tabela_jogos_ajustada["Fora"]]

tabela_jogos_ajustada["gols_casa"] = tabela_jogos_ajustada["Placar"].str.split("–").str[0].astype(int)
tabela_jogos_ajustada["gols_fora"] = tabela_jogos_ajustada["Placar"].str.split("–").str[1].astype(int)
tabela_jogos_ajustada = tabela_jogos_ajustada.drop(columns=["Placar"])  


"""
media_gols_casa_2023 = tabela_jogos_ajustada.groupby("Casa").mean(numeric_only=True)
media_gols_casa_2023 = media_gols_casa_2023.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})

media_gols_fora_2023 = tabela_jogos_ajustada.groupby("Fora").mean(numeric_only=True)
media_gols_fora_2023 = media_gols_fora_2023.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})

# Removendo os times rebaixados
media_gols_casa_2023 = media_gols_casa_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
media_gols_fora_2023 = media_gols_fora_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
"""

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\359129081.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2023 = pd.read_html(seriaA2024.text)


'\nmedia_gols_casa_2023 = tabela_jogos_ajustada.groupby("Casa").mean(numeric_only=True)\nmedia_gols_casa_2023 = media_gols_casa_2023.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})\n\nmedia_gols_fora_2023 = tabela_jogos_ajustada.groupby("Fora").mean(numeric_only=True)\nmedia_gols_fora_2023 = media_gols_fora_2023.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})\n\n# Removendo os times rebaixados\nmedia_gols_casa_2023 = media_gols_casa_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])\nmedia_gols_fora_2023 = media_gols_fora_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])\n'

In [75]:
# Unindo as duas tabelas 2023 - 2024

tabela_jogos_total = pd.concat([tabela_jogos_ajustada, jogos_realizados], ignore_index=True)
media_gols_casa = tabela_jogos_total.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos_casa", "gols_fora": "gols sofridos_casa"})
media_gols_fora = tabela_jogos_total.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos_fora", "gols_fora": "gols feitos_fora"})

media_gols_casa = media_gols_casa.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
media_gols_fora = media_gols_fora.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])


In [76]:
tabela_estatistica = media_gols_casa.merge(media_gols_fora, left_index=True, right_index=True)
tabela_estatistica = tabela_estatistica.reset_index()
tabela_estatistica = tabela_estatistica.rename(columns={"Casa": "Time"})

"""
tabela_estatistica_2023 = media_gols_casa_2023.merge(media_gols_fora_2023, left_index=True, right_index=True)
tabela_estatistica_2023 = tabela_estatistica_2023.reset_index()
tabela_estatistica_2023 = tabela_estatistica_2023.rename(columns={"Casa": "Time"})
"""

'\ntabela_estatistica_2023 = media_gols_casa_2023.merge(media_gols_fora_2023, left_index=True, right_index=True)\ntabela_estatistica_2023 = tabela_estatistica_2023.reset_index()\ntabela_estatistica_2023 = tabela_estatistica_2023.rename(columns={"Casa": "Time"})\n'

## Previsões

In [77]:
from scipy.stats import poisson


def calcular_pts_esperada(linha):
    # Disitribuição de Poisson - Eventos independentes
    time_casa = linha["Casa"]
    time_fora = linha["Fora"]


    # Media esperada de gols que o time da casa tem X contra o time fora de casa
    lambda_casa = tabela_estatistica.loc[tabela_estatistica["Time"] == time_casa, "gols feitos_casa"].iloc[0] * tabela_estatistica.loc[tabela_estatistica["Time"] == time_fora, "gols sofridos_fora"].iloc[0]

    # Media esperada de gols que o time fora de casa tem X contra o time da casa
    lambda_fora = tabela_estatistica.loc[tabela_estatistica["Time"] == time_fora, "gols feitos_fora"].iloc[0] * tabela_estatistica.loc[tabela_estatistica["Time"] == time_casa, "gols sofridos_casa"].iloc[0]


    pv_casa = 0
    p_empate = 0
    pv_fora = 0
    #Considerando fazer gols eventos independentes
    for gols_casa in range(0, 7):
        for gols_fora in range(0, 7):
            prob_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            if gols_casa == gols_fora:
                p_empate += prob_resultado
            elif gols_casa > gols_fora:
                pv_casa += prob_resultado
            else:
                pv_fora += prob_resultado

    v_esperado_casa = pv_casa * 3 + p_empate
    v_esperado_fora = pv_fora * 3 + p_empate

    #print(v_esperado_casa)
    #print(v_esperado_fora)
    linha["pontos_casa"] = v_esperado_casa
    linha["pontos_fora"] = v_esperado_fora
    return linha



In [81]:
tabela_jogos_faltantes = jogos_faltantes.apply(calcular_pts_esperada, axis=1)
tabela_classificação_atualizada = tabela_classificação2024[["Equipevde", "Pts"]]
tabela_classificação_atualizada["Pts"] = tabela_classificação_atualizada["Pts"].astype(int)

pts_casa = tabela_jogos_faltantes.groupby("Casa").sum()[["pontos_casa"]]
pts_fora = tabela_jogos_faltantes.groupby("Fora").sum()[["pontos_fora"]]

def atualizar_pts (linha):
    time = linha["Equipevde"]
    pontuacao = int(linha["Pts"]) + float(pts_casa.loc[time, "pontos_casa"]) + float(pts_fora.loc[time, "pontos_fora"])
    
    return pontuacao

tabela_classificação_atualizada["Pts"] = tabela_classificação_atualizada.apply(atualizar_pts, axis=1)
# Renomear a coluna
tabela_classificação_atualizada = tabela_classificação_atualizada.rename(columns={"Equipevde": "Time", "Pts": "Pontos"})
tabela_classificação_atualizada = tabela_classificação_atualizada.sort_values(by="Pontos", ascending=False).reset_index(drop=True)
tabela_classificação_atualizada.index = tabela_classificação_atualizada.index + 1

#salvando a tabela em json em utf-8
tabela_classificação_atualizada.to_json("classificação.json", orient="records", force_ascii=False)
display(tabela_classificação_atualizada)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\2688318972.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificação_atualizada["Pts"] = tabela_classificação_atualizada["Pts"].astype(int)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_13288\2688318972.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela_classificação_atualizada["Pts"] = tabela_classificação_atualizada.apply(atualizar_pts, axis=1)


,Time,Pontos
1,Botafogo,69.998992
2,Atlético Mineiro,69.446473
3,Palmeiras,68.579209
4,Flamengo,65.405254
5,Athletico Paranaense,61.912518
6,Red Bull Bragantino,61.584967
7,São Paulo,56.851517
8,Bahia,56.575501
9,Cruzeiro,54.795098
10,Internacional,53.474818
